In [1]:
!pip install torch==1.4.0

import torch
print(torch.__version__)
print(torch.cuda.is_available())

     |████████████████████████████████| 753.4 MB 5.5 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 0.4.1
    Uninstalling torch-0.4.1:
      Successfully uninstalled torch-0.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
1.4.0
False


In [2]:
!wget https://www.dropbox.com/s/9narw5x4uizmehh/utils.py
!mkdir images data

# data取得
!wget https://www.dropbox.com/s/o4kyc52a8we25wy/dev.en -P data/
!wget https://www.dropbox.com/s/kdgskm5hzg6znuc/dev.ja -P data/
!wget https://www.dropbox.com/s/gyyx4gohv9v65uh/test.en -P data/
!wget https://www.dropbox.com/s/hotxwbgoe2n013k/test.ja -P data/
!wget https://www.dropbox.com/s/5lsftkmb20ay9e1/train.en -P data/
!wget https://www.dropbox.com/s/ak53qirssci6f1j/train.ja -P data/

--2022-01-03 11:30:11--  https://www.dropbox.com/s/9narw5x4uizmehh/utils.py
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9narw5x4uizmehh/utils.py [following]
--2022-01-03 11:30:12--  https://www.dropbox.com/s/raw/9narw5x4uizmehh/utils.py
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce060bfa4b95c11db57a2061910.dl.dropboxusercontent.com/cd/0/inline/BdFhl-1oFR_cj8l4Ook_N4fFkJwHUok0O0aqq7Qsdcdh7zzzAfd4-swK8LpXBtOOBT0j-yxSsvkiY50W-2dnD_kkJSFe4K6gnFIrc6qfUNRH77ONT68pRO4FclV74WrXHFEl7S4t3V_MuhWFBhp2a-kT/file# [following]
--2022-01-03 11:30:12--  https://uce060bfa4b95c11db57a2061910.dl.dropboxusercontent.com/cd/0/inline/BdFhl-1oFR_cj8l4Ook_N4fFkJwHUok0O0aqq7Qsdcdh7zzzAfd4-swK8LpXBtOOBT0j-yxSsvkiY50W-2dnD_kkJSFe4K6gnFIrc6

In [3]:
!ls data

dev.en	  dev.ja    test.en    test.ja	  train.en    train.ja
dev.en.1  dev.ja.1  test.en.1  test.ja.1  train.en.1  train.ja.1
dev.en.2  dev.ja.2  test.en.2  test.ja.2  train.en.2  train.ja.2


In [4]:
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from nltk import bleu_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from utils import Vocab

# デバイスの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1)
random_state = 42

print(torch.__version__)

1.4.0


1. データセットの準備

In [5]:
!head -10 data/train.en

i can 't tell who will arrive first .
many animals have been destroyed by men .
i 'm in the tennis club .
emi looks happy .
please bear this fact in mind .
she takes care of my children .
we want to be international .
you ought not to break your promise .
when you cross the street , watch out for cars .
i have nothing to live for .


In [6]:
!head -10 data/train.ja

誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
多く の 動物 が 人間 に よ っ て 滅ぼ さ れ た 。
私 は テニス 部員 で す 。
エミ は 幸せ そう に 見え ま す 。
この 事実 を 心 に 留め て お い て 下さ い 。
彼女 は 私 たち の 世話 を し て くれ る 。
私 達 は 国際 人 に な り た い と 思 い ま す 。
約束 を 破 る べ き で は あ り ま せ ん 。
道路 を 横切 る とき は 車 に 注意 し なさ い 。
私 に は 生き 甲斐 が な い 。


1.1 データの読み込みと単語の分割

In [7]:
# テキストファイルからデータを読み込み
def load_data(file_path):
  data = []
  for line in open(file_path, encoding='utf-8'):
    words = line.strip().split()  # スペースで単語を分割
    data.append(words)
  return data

In [8]:
train_X = load_data('./data/train.en')
train_Y = load_data('./data/train.ja')

# 訓練データと検証データに分割
train_X, valid_X, train_Y, valid_Y = train_test_split(train_X, train_Y, test_size=0.2, random_state=random_state)

In [9]:
print(train_X[0])
print(valid_X[0])

['where', 'shall', 'we', 'eat', 'tonight', '?']
['you', 'may', 'extend', 'your', 'stay', 'in', 'tokyo', '.']


1.2 単語辞書の作成

In [10]:
# データセットに登場する各単語にIDを割り振る
# 特殊トークンの定義
PAD_TOKEN = '<PAD>'
BOS_TOKEN = '<S>'
EOS_TOKEN = '</S>'
UNK_TOKEN = '<UNK>'
PAD = 0
BOS = 1
EOS = 2
UNK = 3

In [11]:
MIN_COUNT = 2  # 語彙に含める単語の最低出現回数

word2id = {
    PAD_TOKEN : PAD,
    BOS_TOKEN : BOS,
    EOS_TOKEN : EOS,
    UNK_TOKEN : UNK
}

# 単語辞書を作成
vocab_X = Vocab(word2id=word2id)
vocab_Y = Vocab(word2id=word2id)
vocab_X.build_vocab(train_X, min_count=MIN_COUNT)
vocab_Y.build_vocab(train_Y, min_count=MIN_COUNT)

In [12]:
vocab_size_X = len(vocab_X.id2word)
vocab_size_Y = len(vocab_Y.id2word)
print('入力言語の語彙数：', vocab_size_X)
print('出力言語の語彙数：', vocab_size_Y)

入力言語の語彙数： 3725
出力言語の語彙数： 4405


２ テンソルへの変換

2.１ IDへの変換


In [13]:
# 文章を単語IDのリストに変換
def sentence_to_ids(vocab, sentence):
  ids = [vocab.word2id.get(word, UNK) for word in sentence]
  ids += [EOS]  # EOSを加える
  return ids

In [14]:
train_X = [sentence_to_ids(vocab_X, sentence) for sentence in train_X]
train_Y = [sentence_to_ids(vocab_Y, sentence) for sentence in train_Y]
valid_X = [sentence_to_ids(vocab_X, sentence) for sentence in valid_X]
valid_Y = [sentence_to_ids(vocab_Y, sentence) for sentence in valid_Y]

In [15]:
print(train_X[0])
print(valid_X[0])

[132, 321, 28, 290, 367, 12, 2]
[8, 93, 3532, 36, 236, 13, 284, 4, 2]


2.2 DataLoaderの定義

In [16]:
# 系列が指定の文長になるように末尾をパディング
def pad_seq(seq, max_length):
  res = seq + [PAD for i in range(max_length - len(seq))]
  return res

In [17]:
class DataLoader(object):
  def __init__(self, X, Y, batch_size, shuffle=False):
    '''
    :param X : list, 入力言語の文章（単語IDのリスト）のリスト
    :param Y : list, 出力言語の文章（単語IDのリスト）のリスト
    :param batch_size : int, バッチサイズ
    :param shuffle: bool, サンプルの順番をシャッフルするか否か
    '''
    self.data = list(zip(X, Y))
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.start_index = 0
    self.reset()

  def reset(self):
    if self.shuffle:  # サンプルの順番をシャッフル
      self.data = shuffle(self.data, random_state=random_state)
    self.start_index = 0  # ポインタの位置を初期化
  
  def __iter__(self):
    return self

  def __next__(self):
    # ポインタが最後まで到達したら初期化
    if self.start_index >= len(self.data):
      self.reset()
      raise StopIteration()
    
    # バッチを取得
    seqs_X, seqs_Y = zip(*self.data[self.start_index:self.start_index+self.batch_size])
    # 入力系列seq_Xの文章の長さ順（降順）に系列ペアをソート
    seq_pairs = sorted(zip(seqs_X, seqs_Y), key=lambda p: len(p[0]), reverse=True)
    seqs_X, seqs_Y = zip(*seq_pairs)

    # 短い系列の末尾をパディング
    lengths_X = [len(s) for s in seqs_X]
    lengths_Y = [len(s) for s in seqs_Y]
    max_length_X = max(lengths_X)
    max_length_Y = max(lengths_Y)
    padded_X = [pad_seq(s, max_length_X) for s in seqs_X]
    padded_Y = [pad_seq(s, max_length_Y) for s in seqs_Y]

    # tensor に変換し転置する
    batch_X = torch.tensor(padded_X, dtype=torch.long, device=device).transpose(0,1)
    batch_Y = torch.tensor(padded_Y, dtype=torch.long, device=device).transpose(0,1)

    # ポインタを更新
    self.start_index += self.batch_size

    return batch_X, batch_Y, lengths_X

3. モデルの構築

EncoderとDecoderのRNNを定義



In [18]:
# 系列長がそれぞれ4,3,2の3つのサンプルからなるバッチを作成
batch = [[1,2,3,4], [5,6,7], [8,9]]
lengths = [len(sample) for sample in batch]
print('各サンプルの系列長:', lengths)
print()

# 最大系列長に合うように各サンプルをpadding
_max_length = max(lengths)
padded = torch.tensor([pad_seq(sample, _max_length) for sample in batch])
print('paddingされたテンソル:\n', padded)
padded = padded.transpose(0,1)  # (max_length, batch_size)に転置
print('padding & 転置されたテンソル:\n', padded)
print('padding & 転置されたテンソルのサイズ:\n', padded.size())
print()

各サンプルの系列長: [4, 3, 2]

paddingされたテンソル:
 tensor([[1, 2, 3, 4],
        [5, 6, 7, 0],
        [8, 9, 0, 0]])
padding & 転置されたテンソル:
 tensor([[1, 5, 8],
        [2, 6, 9],
        [3, 7, 0],
        [4, 0, 0]])
padding & 転置されたテンソルのサイズ:
 torch.Size([4, 3])



In [19]:
# PackedSequenceに変換(テンソルをRNNに入力する前に適用する)
packed = pack_padded_sequence(padded, lengths=lengths)
print('PackedSequenceのインスタンス:\n', packed)
print()

PackedSequenceのインスタンス:
 PackedSequence(data=tensor([1, 5, 8, 2, 6, 9, 3, 7, 4]), batch_sizes=tensor([3, 3, 2, 1]), sorted_indices=None, unsorted_indices=None)



In [20]:
# RNNに入力（省略）
output = packed

# テンソルに戻す(RNNの出力に対して適用する)
output, _length = pad_packed_sequence(output)
print(output)
print(_length)

tensor([[1, 5, 8],
        [2, 6, 9],
        [3, 7, 0],
        [4, 0, 0]])
tensor([4, 3, 2])


Encoder

In [21]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size):
    '''
    :param input_size : int, 入力言語の語彙数
    :param hidden_size : 隠れ層のユニット数
    '''
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(input_size, hidden_size, padding_idx=PAD)
    self.gru = nn.GRU(hidden_size, hidden_size)

  def forward(self, seqs, input_lengths, hidden=None):
    '''
    :param seqs : tensor, 入力のバッチ, size=(max_length, batch_size)
    :param input_lengths : 入力のバッチの各サンプルの文長
    :param hidden : tensor, 隠れ状態の初期値, Noneの場合は0で初期化される
    :return output : tensor, Encoderの出力, size=(max_length, batch_size, hidden_size)
    :return hidden : tensor, Encoderの隠れ状態, size=(1, batch_size, hidden_size)
    '''
    emb = self.embedding(seqs)
    packed = pack_padded_sequence(emb, input_lengths)  # PackedSequenceオブジェクトに変換
    output, hidden = self.gru(packed, hidden)
    output, _ = pad_packed_sequence(output)
    return output, hidden

Decorder

In [22]:
class Decoder(nn.Module):
  def __init__(self, hidden_size, output_size):
    '''
    :param hidden_size : 隠れ層のユニット数
    :param output_size : int, 出力言語の語彙数
    :param dropout : float, ドロップアウト率
    '''
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.embedding = nn.Embedding(output_size, hidden_size, padding_idx=PAD)
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)

  def forward(self, seqs, hidden):
    '''
    :param seqs: tensor, 入力のバッチ, size=(1, batch_size)
    :param hidden: tensor, 隠れ状態の初期値, Noneの場合は0で初期化される
    :return output: tensor, Decoderの出力, size=(1, batch_size, output_size)
    :return hidden: tensor, Decoderの隠れ状態, size=(1, batch_size, hidden_size)
    '''
    emb = self.embedding(seqs)
    output, hidden = self.gru(emb, hidden)
    output = self.out(output)
    return output, hidden

EncoderDecoder

In [23]:
class EncoderDecoder(nn.Module):
  '''
  EncoderとDecoderの処理をまとめる
  '''
  def __init__(self, input_size, output_size, hidden_size):
    '''
    :param input_size: int, 入力言語の語彙数
    :param output_size: int, 出力言語の語彙数
    :param hidden_size: int, 隠れ層のユニット数    
    '''
    super(EncoderDecoder, self).__init__()
    self.encoder = Encoder(input_size, hidden_size)
    self.decoder = Decoder(hidden_size, output_size)

  def forward(self, batch_X, lengths_X, max_length, batch_Y=None, use_teacher_forcing=False):
    '''
    :param batch_X: tensor, 入力系列のバッチ, size=(max_length, batch_size)
    :param lengths_X: list, 入力系列のバッチ内の各サンプルの文長
    :param max_length: int, Decoderの最大文長
    :param batch_Y: tensor, Decoderで用いるターゲット系列
    :param use_teacher_forcing: Decoderでターゲット系列を入力とするフラグ
    :return decoder_outputs: tensor, Decoderの出力, 
        size=(max_length, batch_size, self.decoder.output_size)
    '''
    # encoderに系列を入力（複数時刻をまとめて処理）
    _, encoder_hidden = self.encoder(batch_X, lengths_X)
    _batch_size = batch_X.size(1)

    # decoderの入力と隠れ層の初期状態を定義
    decoder_input = torch.tensor([BOS] * _batch_size, dtype=torch.long, device=device)  # 最初の入力にはBOSを使用する
    decoder_input = decoder_input.unsqueeze(0)  # (1, batch_size)
    decoder_hidden = encoder_hidden  # Encoderの最終隠れ状態を取得

    # decoderの出力ホルダーを定義
    decoder_outputs = torch.zeros(max_length, _batch_size, self.decoder.output_size, device=device)

    # 各時刻ごとに処理
    for t in range(max_length):
      decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
      decoder_outputs[t] = decoder_output
      # 次の時刻のdecoderの入力を決定
      if use_teacher_forcing and batch_Y is not None:  # teacher forceの場合、ターゲット系列を用いる
        decoder_input = batch_Y[t].unsqueeze(0)
      else:  # teacher forceでない場合、自身の出力を用いる
        decoder_input = decoder_output.max(-1)[1]

    return decoder_outputs




4. 訓練

4.1 損失関数の定義

In [24]:
mce = nn.CrossEntropyLoss(size_average=False, ignore_index=PAD)  # PADを無視する
def masked_cross_entropy(logits, target):
  logits_flat = logits.view(-1, logits.size(-1))
  target_flat = target.view(-1)
  return mce(logits_flat, target_flat)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


4.2 学習

In [25]:
# ハイパーパラメータの設定
num_epochs = 10
batch_size = 64
lr = 1e-3  # 学習率
teacher_forcing_rate = 0.2  # Teacher Forcingを行う確率
ckpt_path = 'model.pth'  # 学習済みモデルを保存するパス

model_args = {
    'input_size': vocab_size_X,
    'output_size': vocab_size_Y,
    'hidden_size': 256
}

In [26]:
# データローダーを定義
train_dataloader = DataLoader(train_X, train_Y, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_X, valid_Y, batch_size=batch_size, shuffle=False)

# モデルとoptimizerを定義
model = EncoderDecoder(**model_args).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [27]:
# 損失関数を計算する関数
def compute_loss(batch_X, batch_Y, lengths_X, model, optimizer=None, is_train=True):
  model.train(is_train)  # train/evalモードの切り替え

  # 一定確率でTeacher　Forcingを行う
  use_teacher_forcing = is_train and(random.random() < teacher_forcing_rate)
  max_length = batch_Y.size(0)

  # 推論
  pred_Y = model(batch_X, lengths_X, max_length, batch_Y, use_teacher_forcing)
  # 損失関数を計算
  loss = masked_cross_entropy(pred_Y.contiguous(), batch_Y.contiguous())

  if is_train:  # 訓練時はパラメータを更新
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  batch_Y = batch_Y.transpose(0, 1).contiguous().data.cpu().tolist()
  pred = pred_Y.max(dim=-1)[1].data.cpu().numpy().T.tolist()

  return loss.item(), batch_Y, pred

In [28]:
def calc_bleu(refs, hyps):
  '''
  BLEUスコアを計算する関数
  :param refs: list, 参照訳。単語のリストのリスト (例： [['I', 'have', 'a', 'pen'], ...])
  :param hyps: list, モデルの生成した訳。単語のリストのリスト (例： ['I', 'have', 'a', 'pen'])
  :return: float, BLEUスコア(0~100)
  '''
  refs = [[ref[:ref.index(EOS)]] for ref in refs]  # EOSは評価しないで良いので切り捨てる。refsのほうは複数なのでlistが一個多くかかっている
  hyps = [hyp[:hyp.index(EOS)] if EOS in hyp else hyp for hyp in hyps]
  return 100 * bleu_score.corpus_bleu(refs, hyps)

In [29]:
# 訓練
best_valid_bleu = 0.

for epoch in range(1, num_epochs+1):
  train_loss = 0.
  train_refs = []
  train_hyps = []
  valid_loss = 0.
  valid_refs = []
  valid_hyps = []

  # train
  for batch in train_dataloader:
    batch_X, batch_Y, lengths_X = batch
    loss, gold, pred = compute_loss(batch_X, batch_Y, lengths_X, model, optimizer, is_train=True)
    train_loss += loss
    train_refs += gold
    train_hyps += pred
  
  # valid
  for batch in valid_dataloader:
    batch_X, batch_Y, lengths_X = batch
    loss, gold, pred = compute_loss(batch_X, batch_Y, lengths_X, model, is_train=False)
    valid_loss += loss
    valid_refs += gold
    valid_hyps += pred
  
  # 損失をサンプル数で割って正規化
  train_loss = np.sum(train_loss) / len(train_dataloader.data)
  valid_loss = np.sum(valid_loss) / len(valid_dataloader.data)
  # BLEUを計算
  train_bleu = calc_bleu(train_refs, train_hyps)
  valid_bleu = calc_bleu(valid_refs, valid_hyps)

  # validationデータでBLEUが改善した場合にはモデルを保存
  if valid_bleu > best_valid_bleu:
    ckpt = model.state_dict()
    torch.save(ckpt, ckpt_path)
    best_valid_bleu = valid_bleu
  
  print('Epoch {}: train_loss: {:5.2f}  train_bleu: {:2.2f}  valid_loss: {:5.2f}  valid_bleu: {:2.2f}'.format(epoch, train_loss, train_bleu, valid_loss, valid_bleu))
      
  print('-'*80)


Epoch 1: train_loss: 52.28  train_bleu: 3.34  valid_loss: 48.86  valid_bleu: 3.83
--------------------------------------------------------------------------------
Epoch 2: train_loss: 44.90  train_bleu: 7.20  valid_loss: 44.58  valid_bleu: 7.78
--------------------------------------------------------------------------------
Epoch 3: train_loss: 40.62  train_bleu: 10.66  valid_loss: 42.18  valid_bleu: 11.00
--------------------------------------------------------------------------------
Epoch 4: train_loss: 37.50  train_bleu: 13.84  valid_loss: 40.81  valid_bleu: 13.06
--------------------------------------------------------------------------------
Epoch 5: train_loss: 34.88  train_bleu: 16.77  valid_loss: 40.13  valid_bleu: 13.62
--------------------------------------------------------------------------------
Epoch 6: train_loss: 33.04  train_bleu: 18.94  valid_loss: 39.74  valid_bleu: 14.94
--------------------------------------------------------------------------------
Epoch 7: train

5. 評価

In [30]:
# 学習済みモデルの読み込み
ckpt = torch.load(ckpt_path)
model.load_state_dict(ckpt)
model.eval()

EncoderDecoder(
  (encoder): Encoder(
    (embedding): Embedding(3725, 256, padding_idx=0)
    (gru): GRU(256, 256)
  )
  (decoder): Decoder(
    (embedding): Embedding(4405, 256, padding_idx=0)
    (gru): GRU(256, 256)
    (out): Linear(in_features=256, out_features=4405, bias=True)
  )
)

In [31]:
def ids_to_sentence(vocab, ids):
  # IDのリストを単語のリストに変換する
  return [vocab.id2word[_id] for _id in ids]

def trim_eos(ids):
  # IDのリストからEOS以降の単語を除外する
  if EOS in ids:
      return ids[:ids.index(EOS)]
  else:
      return ids

In [32]:
# テストデータの読み込み
test_X = load_data('./data/dev.en')
test_Y = load_data('./data/dev.ja')

In [33]:
test_X = [sentence_to_ids(vocab_X, sentence) for sentence in test_X]
test_Y = [sentence_to_ids(vocab_Y, sentence) for sentence in test_Y]

In [34]:
test_dataloader = DataLoader(test_X, test_Y, batch_size=1, shuffle=False)

In [35]:
# 生成
batch_X, batch_Y, lengths_X = next(test_dataloader)
sentence_X = ' '.join(ids_to_sentence(vocab_X, batch_X.data.cpu().numpy()[:-1, 0]))
sentence_Y = ' '.join(ids_to_sentence(vocab_Y, batch_Y.data.cpu().numpy()[:-1, 0]))
print('src: {}'.format(sentence_X))
print('tgt: {}'.format(sentence_Y))

output = model(batch_X, lengths_X, max_length=20)
output = output.max(dim=-1)[1].view(-1).data.cpu().tolist()
output_sentence = ' '.join(ids_to_sentence(vocab_Y, trim_eos(output)))
output_sentence_without_trim = ' '.join(ids_to_sentence(vocab_Y, output))
print('out: {}'.format(output_sentence))
print('without trim: {}'.format(output_sentence_without_trim))

src: show your own business .
tgt: 自分 の 事 を しろ 。
out: 自分 の 仕事 を を し て い 。 。
without trim: 自分 の 仕事 を を し て い 。 。 </S> </S> </S> </S> </S> </S> </S> </S> </S> </S>


In [36]:
# BLEUを計算
test_dataloader = DataLoader(test_X, test_Y, batch_size=1, shuffle=False)
refs_list = []
hyp_list = []

for batch in test_dataloader:
  batch_X, batch_Y, lengths_X = batch
  pred_Y = model(batch_X, lengths_X, max_length=20)
  pred = pred_Y.max(dim=-1)[1].view(-1).data.cpu().tolist()
  refs = batch_Y.view(-1).data.cpu().tolist()
  refs_list.append(refs)
  hyp_list.append(pred)
bleu = calc_bleu(refs_list, hyp_list)
print(bleu)

19.409227762250016


Beam Seach

In [37]:
class BeamEncoderDecoder(EncoderDecoder):
  '''
  Beam Searchでdecodeを行うためのクラス
  '''
  def __init__(self, input_size, output_size, hidden_size, beam_size=4):
    '''
    :param input_size: int, 入力言語の語彙数
    :param output_size: int, 出力言語の語彙数
    :param hidden_size: int, 隠れ層のユニット数
    :param beam_size: int, ビーム数
    '''
    super(BeamEncoderDecoder, self).__init__(input_size, output_size, hidden_size)
    self.beam_size = beam_size

  def forward(self, batch_X, lengths_X, max_length):
    '''
    :param batch_X: tensor, 入力系列のバッチ, size=(max_length, batch_size)
    :param lengths_X: list, 入力系列のバッチ内の各サンプルの文長
    :param max_length: int, Decoderの最大文長
    :return decoder_outputs: list, 各ビームのDecoderの出力
    :return finished_scores: list of float, 各ビームのスコア
    '''
    _, encoder_hidden = self.encoder(batch_X, lengths_X)

    # decoderの入力と隠れ層の初期状態を定義
    decoder_input = torch.tensor([BOS] * self.beam_size, dtype=torch.long, device=device)
    decoder_input = decoder_input.unsqueeze(0)  # (1, batch_size)
    decoder_hidden = encoder_hidden

    # beam_sizeの数だけrepeatする
    decoder_input = decoder_input.expand(1, beam_size)
    decoder_hidden = decoder_hidden.expand(1, beam_size, -1).contiguous()

    k = beam_size
    finished_beams = []
    finished_scores = []
    prev_probs = torch.zeros(beam_size, 1, dtype=torch.float, device=device)  # 前の時刻の各ビームの対数尤度を保持
    output_size = self.decoder.output_size

    # 各時刻ごとに処理
    for t in range(max_length):
      # decoder_input: (1, k)
      decoder_output, decoder_hidden = self.decoder(decoder_input[-1:], decoder_hidden)
      # decoder_output: (1, k, output_size)
      # decoder_hidden: (1, k, hidden_size)
      decoder_output_t = decoder_output[-1]  # (k, output_size)
      log_probs = prev_probs + F.log_softmax(decoder_output_t, dim=-1)  # (k, output_size)
      scores = log_probs  # 対数尤度をスコアとする

      # スコアの高いビームとその単語を取得
      flat_scores = scores.view(-1)  # (k*output_size,)
      if t == 0:
        flat_scores = flat_scores[:output_size]  # t=0のときは後半の同じ値の繰り返しを除外
      top_vs, top_is = flat_scores.data.topk(k)
      beam_indices = top_is / output_size  # (k,)
      word_indices = top_is % output_size  # (k,)
      
      # ビームを更新する
      _next_beam_indices = []
      _next_word_indices = []
      for b, w in zip(beam_indices, word_indices):
        if w.item() == EOS:  # EOSに到達した場合はそのビームは更新して終了
          k -= 1
          beam = torch.cat([decoder_input.t()[b], w.view(1,)])  # (t+2,)
          score = scores[b, w].item()
          finished_beams.append(beam)
          finished_scores.append(score)
        else:  # それ以外の場合はビームを更新
          _next_beam_indices.append(b)
          _next_word_indices.append(w)
      if k == 0:
        break

      # tensorに変換
      next_beam_indices = torch.tensor(_next_beam_indices, device=device)
      next_word_indices = torch.tensor(_next_word_indices, device=device)

      # 次の時刻のDecoderの入力を更新
      decoder_input = torch.index_select(decoder_input, dim=-1, index=next_beam_indices)
      decoder_input = torch.cat([decoder_input, next_word_indices.unsqueeze(0)], dim=0)

      # 次の時刻のDecoderの隠れ層を更新
      decoder_hidden = torch.index_select(decoder_hidden, dim=1, index=next_beam_indices)

      # 各ビームの対数尤度を更新
      flat_probs = log_probs.view(-1)  # (k*output_size,)
      next_indices = (next_beam_indices + 1) * next_word_indices
      prev_probs = torch.index_select(flat_probs, dim=0, index=next_indices).unsqueeze(1)  # (k, 1)

    # すべてのビームが完了したらデータを整形
    decoder_outputs = [[idx.item() for idx in beam[1:-1]] for beam in finished_beams]
    
    return decoder_outputs, finished_scores

In [38]:
# 学習済みモデルの読み込み
beam_size = 3
beam_model = BeamEncoderDecoder(**model_args, beam_size=beam_size).to(device)
beam_model.load_state_dict(ckpt)
beam_model.eval()

BeamEncoderDecoder(
  (encoder): Encoder(
    (embedding): Embedding(3725, 256, padding_idx=0)
    (gru): GRU(256, 256)
  )
  (decoder): Decoder(
    (embedding): Embedding(4405, 256, padding_idx=0)
    (gru): GRU(256, 256)
    (out): Linear(in_features=256, out_features=4405, bias=True)
  )
)

In [39]:
# テスト用データセットをロード
test_dataloader = DataLoader(test_X, test_Y, batch_size=1, shuffle=False)

In [40]:
# 生成
batch_X, batch_Y, lengths_X = next(test_dataloader)
sentence_X = ' '.join(ids_to_sentence(vocab_X, batch_X.data.cpu().numpy()[:-1, 0]))
sentence_Y = ' '.join(ids_to_sentence(vocab_Y, batch_Y.data.cpu().numpy()[:-1, 0]))
print('src: {}'.format(sentence_X))
print('tgt: {}'.format(sentence_Y))

# 普通のdecode
output = model(batch_X, lengths_X, max_length=20)
output = output.max(dim=-1)[1].view(-1).data.cpu().tolist()
output_sentence = ' '.join(ids_to_sentence(vocab_Y, trim_eos(output)))
print('out: {}'.format(output_sentence))

# beam decode
outputs, scores = beam_model(batch_X, lengths_X, max_length=20)
# scoreの良い順にソート
outputs, scores = zip(*sorted(zip(outputs, scores), key=lambda x: -x[1]))
for o, output in enumerate(outputs):
  output_sentence = ' '.join(ids_to_sentence(vocab_Y, output))
  print('out{}: {}'.format(o+1, output_sentence))

src: show your own business .
tgt: 自分 の 事 を しろ 。
out: 自分 の 仕事 を を し て い 。 。
out1: 自分 の 持ち物 に <UNK> し て い 。
out2: 自分 の 持ち物 に <UNK> し て い い 。
out3: 自分 の 持ち物 に <UNK> し て い 。 。
